In [201]:
!pip install beautifulsoup4 -q
!pip install lxml -q
!pip install geocoder
!conda install -c conda-forge folium=0.5.0 --yes

\ ^C
failed

CondaError: KeyboardInterrupt



In [205]:
import numpy
import pandas
import requests
from bs4 import BeautifulSoup
import geocoder
import folium

In [206]:
!wget -q -O 'wikipedia_canada_postal_codes.html' 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
!wget -q -O 'canada_geodata.csv' 'https://cocl.us/Geospatial_data'

In [210]:
canada_geodata = pandas.read_csv('canada_geodata.csv')

def joinstr(x):
    return pandas.Series(dict(Neighbourhood = "%s" % ', '.join(x['Neighbourhood'])))

postal_codes = pandas.DataFrame(columns=['PostalCode', 'Borough', 'Neighbourhood'])
with open('wikipedia_canada_postal_codes.html') as html_file:
    soap = BeautifulSoup(html_file)
    postalcode_table = soap.table
    rows = iter(postalcode_table.find_all('tr'))
    next(rows)
    for postal_row in rows:
        postalcode = postal_row.contents[1].text.strip()
        borough = postal_row.contents[3].text.strip()
        neighbourhood = postal_row.contents[5].text.strip()
        if 'assigned' in borough:
            continue
        if  'assigned' in neighbourhood:
            neighbourhood = borough
        postal_codes = postal_codes.append({  'PostalCode': postalcode, 'Borough': borough, 'Neighbourhood': neighbourhood}, ignore_index=True)
postal_codes = postal_codes.groupby(['PostalCode', 'Borough']).apply(joinstr).reset_index()

def get_lat(postal_code):
    return canada_geodata[canada_geodata['Postal Code'] == postal_code].iat[0, 1]

def get_lng(postal_code):
    return canada_geodata[canada_geodata['Postal Code'] == postal_code].iat[0, 2]

# Create the Latitude and Longitude columns based on the Postal Code
postal_codes['Latitude'] = postal_codes['PostalCode'].apply(get_lat)
postal_codes['Longitude'] = postal_codes['PostalCode'].apply(get_lng)
toronto = postal_codes[postal_codes['Borough'].str.contains("Toronto")]
toronto

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
49,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


In [214]:
map_toronto = folium.Map(location=[43.651070, -79.347015], zoom_start=12)

# add markers to map
for lat, lng, borough in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto